In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
from LumFunc_miniJPAS import LumFunc_hist

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
split_converter = lambda s: np.array(s.split()).astype(float)
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))

cat = pd.read_csv('csv/minijpas.Flambda_photoz_gaia_7.csv', sep=',', header=1,
    converters={0: int, 1: int, 2: split_converter, 3: split_converter, 4: sum_flags,
    5: sum_flags})

cat = cat[np.array([len(x) for x in cat['FLUX_AUTO']]) != 0] # Drop bad rows due to bad query
cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
cat = cat.reset_index()

pm_flx = np.stack(cat['FLUX_AUTO'].to_numpy()).T * 1e-19
pm_err = np.stack(cat['FLUX_RELERR_AUTO'].to_numpy()).T * pm_flx

In [ ]:
w_lya = 1215.67 # A

N_sources = len(cat['FLUX_AUTO'])
z_nb_Arr = w_central[:-4] / w_lya - 1
print(N_sources)

In [ ]:
pm_mask = mask_proper_motion(cat)

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 30, mask=pm_mask)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    15, mask=pm_mask)
other_lines = identify_lines(line_other, pm_flx, pm_err)

In [ ]:
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])
nice_lya = nice_lya_select(lya_lines, other_lines, pm_flx, cont_est_other, z_Arr)

In [ ]:
total_hist = np.zeros(15)
volume = 0
for nb_c in range(0, 25):
    LAE_src = np.array(lya_lines) == nb_c
    flambda = pm_flx[nb_c, nice_lya & LAE_src] - cont_est_lya[nb_c, nice_lya & LAE_src]

    LF_bins, LF_hist, volume_i, bin_width = LumFunc_hist(
        flambda, w_central[nb_c], nb_fwhm_Arr[nb_c],
        n_bins=15, L_min=43.5, L_max=45
    )
    total_hist += LF_hist
    volume += volume_i

In [ ]:
## Load HETDEX LumFunc data (Zhang 2021)
filepath = 'csv/HETDEX_LumFunc.csv'
HETDEX_LF = []
with open(filepath, mode='r') as csvfile:
    rdlns = csv.reader(csvfile, delimiter=',')
    for line in rdlns:
        HETDEX_LF.append(line)
HETDEX_LF = np.array(HETDEX_LF).astype(float)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.errorbar(LF_bins, total_hist / volume / bin_width,
    yerr= total_hist ** 0.5 / volume / bin_width,
    marker='.', linestyle='', markersize=15, label='Our data')
ax.errorbar(HETDEX_LF[:,0], HETDEX_LF[:,1], yerr=HETDEX_LF[:, 2] - HETDEX_LF[:, 1],
    fmt='s', markersize = 10, label = 'Zhang (2021)', zorder=-99)
ax.plot(np.linspace(43.5, 45, 100), np.ones(100) / volume / bin_width)

ax.set_yscale('log')
ax.set_xlabel(r'$L_{\mathrm{Ly}\alpha}$ (erg s$^{-1}$)', fontsize=15)
ax.set_ylabel(r'$\Phi$ (Mpc$^{-3}\,\Delta\logL^{-1}$)',
    fontsize=15)
ax.legend(fontsize=15)

plt.show()